In [1]:
import random

In [32]:
class Deck:
    def __init__(self):
        self.deck = []
        suites = ["♥", "♦", "♣", "♠"]
        cards = [2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K", "A"]
        for s in suites:
            for c in cards:
                self.deck.append(str(c) + s)
        random.shuffle(self.deck)
    
    def __str__(self):
        return str(self.deck)
    
    def __len__(self):
        return len(self.deck)
    
    def __call__(self, n):
        cards = self.deck[:n]
        self._remove(n)
        return cards         
    
    def _remove(self, n):
        self.deck = self.deck[n:]

In [33]:
class Game:
    def __init__(self, *players):
        self.deck = None
        self.board = []
        self.bets = {}
        self.players = players
        self.pot = 0
             
    def play(self):
        if len(self.players) < 2:
            print("Need more players.")
            return
        
        while True:
            self.deck = Deck()
            self.board = []
            self.hand()
            
            temp_players = []
            for player in self.players:
                if player.money == 0:
                    print(f"You're out {player.name}!")
                else:
                    temp_players.append(player)
            
            self.players = temp_players
            if len(self.players) == 1:        
                break
            break
            
    def reset_bets(self):
        for player in self.players:
            self.bets[player.name] = -1
            
    def hand(self):
        for player in self.players:
            self.bets[player.name] = -1
            player.hand = []
            player.is_playing = True
            print(player)
        
        self.deal()
        
        # play flop, turn, river
        self._play(3)
        self._play(1)
        self._play(1)
        
        self.find_winner()
        
    def _play(self, n):
        self.board += self.deck(n)
        self.bet()
        self.raises()
        
        for k in self.bets:
            if self.bets[k] > 0:
                self.pot += self.bets[k]
        
        self.is_over()
        self.reset_bets()
        
    def deal(self):
        for player in self.players:
            player.hand = self.deck(2)    
            
    def is_over(self):
        still_playing = 0
        for player in self.players:
            if player.is_playing:
                still_playing += 1
        if still_playing == 1:
            return True
        return False
                
    def bet(self,):
        highest_bet = 0
        for player in self.players:
            if player.is_playing and not self.is_over():
                
                #### MAIN MOVE ####
                bet = player.move(self.board, self.pot, self.bets, self.players) 
                ###################
                
                if bet > highest_bet:
                    highest_bet = bet
                if player.is_playing:
                    self.bets[player.name] = bet
                
    def raises(self):
        highest_bet = max(self.bets.values())
        print("Highest bet", highest_bet)
        for player in self.players:
            difference = highest_bet - self.bets[player.name]
            if player.is_playing and self.bets[player.name] < highest_bet and player.money >= difference:
                if player.call(difference):
                    self.bets[player.name] = highest_bet
    
    def find_winner(self):
        winner = None
        for player in self.players:
            # rank hand
            winner = player
        print(f"{winner.name} wins {self.pot}!\n")
        winner.money += self.pot
        self.pot = 0

In [45]:
class Player:
    def __init__(self, name, money=100):
        self.name = name
        self.money = money
        self.hand = []
        self.is_playing = True
        
    def __str__(self):
        return f"Player {self.name}. Hand {self.hand}. Money {self.money}"
    
    def move(self, board, pot, bets, players):
        print("\n")
        
        print("Pot", pot)
        print("Board:", board)
        print("Hand:", self.hand)
        print("Money:", self.money)
        for player in players:
            if player is not self:
                print(f"{player.name}: Bet {bets[player.name]}. Money: {player.money}")
            
        move = 9999
        while move > self.money:
            move = int(input(f"{self.name}: bet? (-1 fold, 0 check): "))
        
        if move < 0:
            self.fold()
            return -1
        
        self.money -= move
        return move
    
    def call(self, amount):
        print("\nMoney:", self.money)
        move = input(f"{self.name} call for +{amount}? (y/n) ")
        if move == "n":
            self.fold()
            return False
        else:
            self.money -= amount
            return True
    
    def fold(self):
        self.is_playing = False

In [46]:
p1 = Player("Austin")
p2 = Player("James")

In [47]:
game = Game(p1, p2)

In [48]:
game.play()

Player Austin. Hand []. Money 100
Player James. Hand []. Money 100


Pot 0
Board: ['Q♦', '8♦', '8♣']
Hand: ['2♣', '10♠']
Money: 100
James: Bet -1. Money: 100


Austin: bet? (-1 fold, 0 check):  -1


Highest bet -1
Highest bet -1
Highest bet -1
James wins 0!

